In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm
sns.set()


In [ ]:
dick = pd.read_pickle('country_data.pkl')
world_df = pd.read_pickle('global_data.pkl')

swe = dick['Sweden']
swe

In [ ]:
M_mu_s = pm.rbeta(alpha=2,beta=100,size=100000)
_ = plt.hist(M_mu_s)

In [ ]:
F_mu_s = pm.rlognormal(1,1/0.2**2, size=1000000)
_ = plt.hist(F_mu_s,bins=50)

F_mu_s.min()

In [ ]:


def process_country(country,C,D):
    df = dick[country]
    
    mask = df['confirmed'] > 1000
    df = df[mask]
    mask = df['deceased'] > 5
    df = df[mask]
    
    confirmed_mean = df['confirmed'].mean()
    confirmed_std = df['confirmed'].std()
    dead_mean = df['deceased'].mean()
    dead_std = df['deceased'].std()
    
    #confirmed_s = (df['confirmed'] - confirmed_mean) / confirmed_std
    #dead_s = (df['deceased'] - dead_mean) / dead_std
    
    confirmed_s = df[C]
    dead_s = df[D]
    
    plt.plot(confirmed_s,dead_s,'x--')
    
    #M_mu = pm.Uniform('M_mu',lower=0.1,upper=1)
    
    F_mu = pm.Uniform('F_mu',lower=1,upper=1.2)
    F_sigma = pm.Uniform('F_sigma',lower=0.19,upper=0.21)
    
    M_alpha = pm.Uniform('M_alpha',lower=1,upper=1000)
    M_beta = pm.Uniform ('M_beta',lower=1,upper=1000)
    
    
    M = pm.Beta('M',alpha=M_alpha,beta=M_beta)
    F = pm.Lognormal('F',mu=F_mu, tau=1 / F_sigma ** 2) #0.6 0.2
    
    
    @pm.deterministic()
    def gen_dead(confirmed_s=confirmed_s,F=F,M=M):
        return confirmed_s * F * M
    
    obs = pm.Poisson('obs',mu=gen_dead,observed=True,value=dead_s)
    
    model = pm.Model([M_alpha,M_beta,F_mu,F_sigma,M,F,gen_dead,obs])
    map_ = pm.MAP(model)
    try:
        map_.fit()
    except:
        print ('couldnt fit {}'.format(country))
        
    mcmc = pm.MCMC(model)
    mcmc.sample(1000000,500000,2)
    
    M_alpha_post = mcmc.trace('M_alpha')[:]
    M_beta_post = mcmc.trace('M_beta')[:]
    F_mu_post = mcmc.trace('F_mu')[:]
    F_sigma_post = mcmc.trace('F_sigma')[:]
    M_post = mcmc.trace('M')[:]
    F_post = mcmc.trace('F')[:]
    
    result = pd.DataFrame({'M_alpha_post' : M_alpha_post,
                          'M_beta_post' : M_beta_post,
                          'F_mu_post' : F_mu_post,
                          'F_sigma_post' : F_sigma_post,
                          'M_post' : M_post,
                          'F_post' : F_post})
    
    return result

def plot(country_name,result):
    
    M_low,M_high = np.percentile(result.M_post,[5,94])
    F_low,F_high = np.percentile(result.F_post,[5,94])
    
    plt.figure(figsize=(18,12))
    plt.subplot(211)
    plt.title('Bayesian estimation of Mortality rate for {}'.format(country_name))
    plt.xlabel('Mortality rate')
    plt.ylabel('Density')
    plt.axvline(M_low,color='orange',ls='dashed',label='89% CI lower')
    plt.axvline(M_high,color='orange',ls='dashed',label='89% CI upper')
    plt.legend(loc='upper left')
    plt.hist(result.M_post,density=True,bins=10)

    plt.subplot(212)
    plt.title('Bayesian estimation of "Off-By-Factor for Confirmed" rate for {}'.format(country_name))
    plt.xlabel('Factor for Confirmed')
    plt.ylabel('Density')
    plt.axvline(F_low,color='orange',ls='dashed',label='89% CI lower')
    plt.axvline(F_high,color='orange',ls='dashed',label='89% CI upper')
    plt.legend(loc='upper left')
    plt.hist(result.F_post,density=True,bins=10)

    plt.savefig('Bayesian_estimate_mortality_and_factor_{}_.jpg'.format(country_name),format='jpg')


country_list = ['China','Italy','Spain','US','Korea, South', 'Sweden','Germany','United Kingdom']

for c in country_list:
    result = process_country(c,'confirmed','deceased')
    plot(c,result)
    print ()
    print (result.describe())



In [ ]:
def process_world(df,C,D):
    
    mask = df['confirmed'] > 1000
    df = df[mask]
    mask = df['dead'] > 5
    df = df[mask]
    
    confirmed_mean = df['confirmed'].mean()
    confirmed_std = df['confirmed'].std()
    dead_mean = df['dead'].mean()
    dead_std = df['dead'].std()
    
    #confirmed_s = (df['confirmed'] - confirmed_mean) / confirmed_std
    #dead_s = (df['deceased'] - dead_mean) / dead_std
    
    confirmed_s = df[C]
    dead_s = df[D]
    
    plt.plot(confirmed_s,dead_s,'x--')
    
    M_mu = pm.Uniform('M_mu',lower=0.1,upper=1)
    M_sigma = pm.Uniform('M_sigma',lower=0,upper=1)
    F_mu = pm.Uniform('F_mu',lower=1,upper=5)
    F_sigma = pm.Uniform('F_sigma',lower=0,upper=0.1)
    
    M = pm.Lognormal('M',mu=M_mu,tau=1/M_sigma ** 2)
    F = pm.Lognormal('F',mu=F_mu,tau=1/F_sigma ** 2)
    
    
    @pm.deterministic()
    def gen_dead(confirmed_s=confirmed_s,F=F,M=M):
        return confirmed_s * F * M
    
    obs = pm.Poisson('obs',mu=gen_dead,observed=True,value=dead_s)
    
    model = pm.Model([M_mu,M_sigma,F_mu,F_sigma,M,F,gen_dead,obs])
    map_ = pm.MAP(model)
    map_.fit()
    mcmc = pm.MCMC(model)
    mcmc.sample(1000000,50000,2)
    
    M_mu_post = mcmc.trace('M_mu')[:]
    M_sigma_post = mcmc.trace('M_sigma')[:]
    F_mu_post = mcmc.trace('F_mu')[:]
    F_sigma_post = mcmc.trace('F_sigma')[:]
    M_post = mcmc.trace('M')[:]
    F_post = mcmc.trace('F')[:]
    
    result = pd.DataFrame({'M_mu_post' : M_mu_post,
                          'M_sigma_post' : M_sigma_post,
                          'F_mu_post' : F_mu_post,
                          'F_sigma_post' : F_sigma_post,
                          'M_post' : M_post,
                          'F_post' : F_post})
    
    return result

def plot_world(result,title):
    
    M_low,M_high = np.percentile(result.M_post,[5,94])
    F_low,F_high = np.percentile(result.F_post,[5,94])
    
    plt.figure(figsize=(18,12))
    plt.subplot(211)
    plt.title('Bayesian estimation of Mortality rate for {}'.format(title))
    plt.xlabel('Mortality rate')
    plt.ylabel('Density')
    plt.axvline(M_low,color='orange',ls='dashed',label='89% CI lower')
    plt.axvline(M_high,color='orange',ls='dashed',label='89% CI upper')
    plt.legend(loc='upper left')
    plt.hist(result.M_post,density=True,bins=10)

    plt.subplot(212)
    plt.title('Bayesian estimation of "Off-By-Factor for Confirmed" rate for {}'.format(title))
    plt.xlabel('Factor for Confirmed')
    plt.ylabel('Density')
    plt.axvline(F_low,color='orange',ls='dashed',label='89% CI lower')
    plt.axvline(F_high,color='orange',ls='dashed',label='89% CI upper')
    plt.legend(loc='upper left')
    plt.hist(result.F_post,density=True,bins=10)

    plt.savefig('Bayesian_estimate_mortality_and_factor_{}.jpg'.format(title),format='jpg')

world_df = pd.read_pickle('global_data.pkl')

result_world = process_world(world_df,'confirmed','deceased')
plot_world(result_world,'Global')